In [ ]:
import random
import math
import numpy as np

In [ ]:
import time
from collections import deque
import copy
import os

import torch
from torch.distributions import Normal
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# tensorboard用
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

# GPUが使える環境なら使う
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
cpu


In [ ]:
# Google DriveをColabにマウント
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


work_dir = '/content/drive/MyDrive/masa/'
os.chdir(work_dir)

os.makedirs('./Alpha_pytorch_model/', exist_ok=True)  # フォルダがない時は生成

Mounted at /content/drive


## 1.ゲーム系

### 1.1 ゲームの作成

学習させたいゲームを選んで実行

#### 1.1.1 三目並べ

In [ ]:
class State:  # ゲーム状態
    # 初期化
    def __init__(self, pieces=None, enemy_pieces=None):
        # 石の配置
        self.pieces = pieces if pieces != None else [0] * 9
        self.enemy_pieces = enemy_pieces if enemy_pieces != None else [0] * 9

    # 石の数の取得
    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count += 1
        return count

    # 負けかどうか
    def is_lose(self):
        # 3並びかどうか
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or \
                        self.enemy_pieces[x+y*3] == 0:
                    return False
                x, y = x+dx, y+dy
            return True

        # 負けかどうか
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1):
            return True
        for i in range(3):
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False

    # 引き分けかどうか
    def is_draw(self):
        return self.piece_count(self.pieces) + self.piece_count(self.enemy_pieces) == 9

    # ゲーム終了かどうか
    def is_done(self):
        return self.is_lose() or self.is_draw()

    # 次の状態の取得
    def next(self, action):
        pieces = self.pieces.copy()
        pieces[action] = 1
        return State(self.enemy_pieces, pieces)

    # 合法手のリストの取得
    def legal_actions(self):
        actions = []
        for i in range(9):
            if self.pieces[i] == 0 and self.enemy_pieces[i] == 0:
                actions.append(i)
        return actions

    # 先手かどうか
    def is_first_player(self):
        return self.piece_count(self.pieces) == self.piece_count(self.enemy_pieces)

    # 文字列表示
    def __str__(self):
        ox = ('o', 'x') if self.is_first_player() else ('x', 'o')
        str = ''
        for i in range(9):
            if self.pieces[i] == 1:
                str += ox[0]
            elif self.enemy_pieces[i] == 1:
                str += ox[1]
            else:
                str += '-'
            if i % 3 == 2:
                str += '\n'
        return str

#### 1.1.2 はじめに指を11,11として、指を選択して足していくゲーム

In [ ]:
class State:
    def __init__(self, hand=None, enemy_hand=None, count=0, max_count=50):
        self.hand = hand if hand != None else [1, 1]  # 自分の手
        self.enemy_hand = enemy_hand if enemy_hand != None else [1, 1]  # 相手の手
        self.count = count if count != 0 else 0  # 手数
        self.max_count = max_count  # 引き分けまでの手数

    # 負けかどうか
    def is_lose(self):
        # 自分の手の合計が0 かつ 相手の手の合計が0以外
        return sum(self.hand)==0 and sum(self.enemy_hand)!=0

    # 引き分けかどうか
    def is_draw(self):
        # 手数が上限値 かつ 両者の手が共に0以外
        return self.count == self.max_count and sum(self.hand)*sum(self.enemy_hand) != 0
    
    # ゲーム終了かどうか
    def is_done(self):
        return self.is_lose() or self.is_draw()

    # 合法手の攻撃リストの取得
    def legal_attacks(self):
        attacks = []
        for i in range(2):
            for j in range(2):
                # 0じゃない手から0じゃない手に攻撃は可能
                if self.hand[i] != 0 and self.enemy_hand[j] != 0:
                    attacks.append(2*i+j)  # 左から左は0、左から右は1、右から左は2、右から右は3
        return attacks

    # 合法手の分身リストの取得
    def legal_splits(self):
        splits = []
        hand_10 = self.hand[0]*10+self.hand[1]  # 10進数表記
        if hand_10 == 2 or hand_10 == 20:
            splits.append(11)
        if hand_10 == 3 or hand_10 == 30:
            splits.append(12)
            splits.append(21)
        if hand_10 == 4 or hand_10 == 40:
            splits.append(13)
            splits.append(22)
            splits.append(31)
        return splits

    # 合法手のリストの取得
    def legal_actions(self):
        return self.legal_attacks()+self.legal_splits()  # 足し算でできる

    # 次の状態の取得
    def next(self, action):
        # 行動を実行するときに手数を1増やして, 先手後手を入れ替える
        state = State(self.hand.copy(), self.enemy_hand.copy(), self.count+1)
        if action > 10:
            state.hand = [action//10, action%10]
        else:
            state.enemy_hand[action%2] += state.hand[action//2]
            state.enemy_hand[action%2] %= 5
        w = state.hand
        state.hand = state.enemy_hand
        state.enemy_hand = w
        return state
    
    # コメント表示
    def comment(self, action):
        str = '{}手目:'.format(self.count)
        str += '先手のターン...' if self.is_first_player() else '後手のターン...'
        if action>10:
            str += '{}と{}に分身'.format(action//10, action%10)
        else:
            dic = ['左', '右']
            str += '{}から{}へ攻撃'.format(dic[action//2], dic[action%2])
        return str

    # 先手かどうか
    def is_first_player(self):
        # 2で割った余りが0のとき先手, 0手目は先手
        return self.count%2 == 0

    # 文字列表示
    def __str__(self):
        # 先手を下に表示
        # 1回目の出力では先手ではないけど, 手が入れ替わっているのであっている.
        up = self.enemy_hand if self.is_first_player() else self.hand
        down = self.hand if self.is_first_player() else self.enemy_hand
        return '{}\n{}'.format(up, down)

### 1.2 行動選択の3つの方法

#### 1.2.1 ランダム

In [ ]:
def random_action(state):  # ランダムで行動選択
    legal_actions = state.legal_actions()
    return legal_actions[random.randint(0, len(legal_actions)-1)]

#### 1.2.2 アルファベータ法

In [ ]:
# アルファベータ法で状態価値を計算
def alpha_beta(state, alpha, beta, depth, max_depth=20):
    # depthは深さを表す. なんて先まで読むかを指定できる.
    # print(depth)
    if state.is_lose():
        return -1
    if state.is_draw():
        return 0
    if depth == max_depth:
        return 0

    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -beta, -alpha, depth+1, max_depth)
        if score > alpha:
            alpha = score
        if alpha >= beta:
            return alpha
    return alpha

# アルファベータ法で行動選択
def alpha_beta_action(state, max_depth=20):
    """
    max_depth: 何手先まで予測するか
    """
    best_action = 0
    alpha = -float('inf')
    actions = []
    scores = []
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -float('inf'), -alpha, 1, max_depth=max_depth)
        # 始めの深さは1
        if score > alpha:
            best_action = action
            alpha = score

        actions.append(action)
        scores.append(score)  # score:1は勝ち確定, score:0は相手がミスしないと引き分け, score:-1は相手がミスしないと負け確定
    # print(actions, scores)
    if max(scores) == 0:  # 最大スコアが0のときは, 0のものの中でランダムに選択
        actions = np.array(actions)
        scores = np.array(scores)
        actions_0 = actions[scores==0]
        best_action = actions_0[random.randint(0, len(actions_0)-1)]

    return best_action

#### 1.2.3 モンテカルロ木探索

In [ ]:
# モンテカルロ木探索用のランダムプレイアウト
def playout(state):
    if state.is_lose():
        return -1
    if state.is_draw():
        return 0
    return -playout(state.next(random_action(state)))


# モンテカルロ木探索で行動選択
def mcts_action(state, expansion=10, evaluation=100):
    """
    expansion: 展開するまでの評価の回数
    evaluation: プレイアウトの試行回数
    """
    class Node:  # モンテカルロ木探索のノードの定義
        def __init__(self, state):
            self.state = state
            self.w = 0  # 累計価値
            self.n = 0  # 試行回数
            self.child_nodes = None

        def evaluate(self):  # evaluate：評価
            if self.state.is_done():  # ノードがゲーム終了まで行った場合。プレイアウトの報酬と一緒。
                # 相手が置いて自分のターンになったとき判定するので、負けか引き分けかしかない。
                value = -1 if self.state.is_lose() else 0

                self.w += value
                self.n += 1
                return value

            if not self.child_nodes:  # 一番下の葉のときplayout
                value = playout(self.state)

                self.w += value
                self.n += 1

                if self.n == expansion:  # 試行回数が10回をこえたら子ノードの展開
                    self.expand()  # 下で定義した関数へ
                return value

            else:  # まだ下に葉が存在するとき、UCB1で求めた一つ下のノードでもう一回この関数を実行
                value = -self.next_child_node().evaluate()  # 下で定義した関数へ。この前にあるマイナスがポイント！

                self.w += value  # 下のノードの価値の探索が終わったら、その価値の逆を上のノードの価値に伝播させる。
                self.n += 1
                return value

        def expand(self):  # 子ノードの作成、上で使われる
            legal_actions = self.state.legal_actions()
            self.child_nodes = []
            for action in legal_actions:
                self.child_nodes.append(Node(self.state.next(action)))

        def next_child_node(self):  # 下のどのノードを探索するか、上で使われる
            for child_node in self.child_nodes:  # 試行回数が0のものは優先的に
                if child_node.n == 0:
                    return child_node

            t = 0  # t：全ての行動の試行回数
            for c in self.child_nodes:
                t += c.n
            ucb1_value = []
            for child_node in self.child_nodes:
                # 一項目のマイナスがポイント！子ノードは敵の価値なのでマイナス必要
                ucb1_value.append(-child_node.w/child_node.n + (2*np.log(t)/child_node.n)**0.5)

            # どの行動をUCB1で探索しているかのデバッグ用
            # legal_actions = self.state.legal_actions()
            # print(legal_actions[np.argmax(ucb1_value)], end=' -> ')

            return self.child_nodes[np.argmax(ucb1_value)]

    root_node = Node(state)  # 今の局面のノードを作成
    root_node.expand()  # 一つ下は必ず探索する

    for _ in range(evaluation):  # 100回探索する
        root_node.evaluate()
        # print()

    legal_actions = state.legal_actions()
    n_list = []
    for c in root_node.child_nodes:  # 外部でクラス内の数値を呼ぶとき
        n_list.append(c.n)
    
    # print('a_list:', legal_actions)  # どの行動が取れるか
    # print('n_list:', n_list)  # 上の行動を100回のうち何回おこなったか
    # print('action:', legal_actions[np.argmax(n_list)])  # 結果どの行動をとるか
    return legal_actions[np.argmax(n_list)]

### 1.3 ゲームのお試し

#### 1.3.1 一回だけ対戦

In [ ]:
state = State()
while True:
    if state.is_done():
        if state.is_lose() and state.is_first_player():
            print('後手の勝ち')
        elif state.is_lose():
            print('先手の勝ち')
        else:
            print('引き分け')
        break

    if state.is_first_player():
        action = random_action(state)  # ランダム
        # action = alpha_beta_action(state, max_depth=2)  # アルファベータ法
        # action = mcts_action(state, expansion=10, evaluation=100)  # モンテカルロ木探索
    else:
        # action = random_action(state)  # ランダム
        # action = alpha_beta_action(state, max_depth=20)  # アルファベータ法
        action = mcts_action(state, expansion=10, evaluation=100)  # モンテカルロ木探索

    state = state.next(action)
    print(state)
    print()

---
---
o--


---
-x-
o--


---
-xo
o--


---
-xo
ox-


-o-
-xo
ox-


xo-
-xo
ox-


xoo
-xo
ox-


xoo
-xo
oxx


後手の勝ち


#### 1.3.2 複数回対戦

In [ ]:
ALL_GAME = 100

    points=[]
    for i in range(ALL_GAME):

        state = State()
        while True:
            if state.is_done():
                break
            if state.is_first_player():  # 対戦ルールを指定
                # action = random_action(state)  # ランダム
                # action = alpha_beta_action(state, max_depth=10)  # アルファベータ法
                action = mcts_action(state, expansion=10, evaluation=100)  # モンテカルロ木探索
            else:
                action = alpha_beta_action(state, max_depth=5)  # アルファベータ法

            state = state.next(action)


        if state.is_lose():
            point = [0, 1, 0] if state.is_first_player() else [1, 0, 0]
        else:
            point = [0, 0, 1]
        points.append(point)

        print('\r試合数 {}/{}'.format(i + 1, ALL_GAME), end='')
    print()

    all_point = np.sum(points, axis=0)

    print('先手{}勝{}敗{}分'.format(all_point[0], all_point[1], all_point[2]))

## 2.モデル

In [ ]:
class Dual(nn.Module):
    def __init__(self, hidden_dim=128):
        super().__init__()
        self.input_dim = 3*3*2
        self.action_dim = 9
        self.fc1 = nn.Linear(self.input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3a = nn.Linear(hidden_dim, self.action_dim)  # 方策用
        self.fc3b = nn.Linear(hidden_dim, 1)  # 価値用

    def forward(self, input):
        hidden = F.relu(self.fc1(input))
        hidden = F.relu(self.fc2(hidden))
        p = F.softmax(self.fc3a(hidden), dim=-1)
        v = torch.tanh(self.fc3b(hidden))
        return p, v


## 3.Agent

### 3.1 行動決定

In [ ]:
PV_EVALUATE_COUNT = 50  # 1推論あたりのシミュレーション回数（本家は1600）

def predict(model, state):
    x = torch.as_tensor([state.pieces, state.enemy_pieces], dtype=torch.float, device=device).view(-1, 18)
    p, v = model(x)

    p = p[0][list(state.legal_actions())].detach().numpy()
    v = v[0][0].detach().numpy()

    return p, v


def nodes_to_scores(nodes):  # ノードのリストを試行回数のリストに変換
    scores = []
    for c in nodes:
        scores.append(c.n)
    return scores  # [10, 15, ...]みたな訪問回数


def pv_mcts_scores(model, state, temperature):
    class Node:  # モンテカルロ木探索のノードの定義
        def __init__(self, state, p):  # クラスの定義に方策も追加
            self.state = state  # 状態
            self.p = p  # 方策
            self.w = 0  # 累計価値
            self.n = 0  # 試行回数
            self.child_nodes = None

        def evaluate(self):
            # ゲーム終了時
            if self.state.is_done():  # ノードがゲーム終了まで行った場合。プレイアウトの報酬と一緒。
                value = -1 if self.state.is_lose() else 0  # 勝敗結果で価値を取得

                self.w += value
                self.n += 1
                return value

            if not self.child_nodes:  # 一番下の葉のとき
                policies, value = predict(model, self.state)  # playoutで求めていた価値をモデルから計算

                self.w += value
                self.n += 1

                # predictした瞬間、即展開。predictで1度価値を求めたら以後ここには来ない。
                self.child_nodes = []
                for action, policy in zip(self.state.legal_actions(), policies):
                    self.child_nodes.append(Node(self.state.next(action), policy))  # 方策とセット
                return value

            else:  # まだ下に葉が存在するとき
                value = -self.next_child_node().evaluate()  # アーク評価値で枝をつたり、その価値を伝播

                self.w += value
                self.n += 1
                return value

        def next_child_node(self):  # アーク評価値が最大の子ノードを取得
            C_PUCT = 1.0
            t = sum(nodes_to_scores(self.child_nodes))
            pucb_values = []
            for child_node in self.child_nodes:
                pucb_values.append((-child_node.w / child_node.n if child_node.n else 0.0) +
                                    C_PUCT * child_node.p * np.sqrt(t) / (1 + child_node.n))

            return self.child_nodes[np.argmax(pucb_values)]

    # ここからmain
    root_node = Node(state, 0)  # 今の局面のノードを作成

    for _ in range(PV_EVALUATE_COUNT):  # ルートノードを100回探索
        root_node.evaluate()

    scores = nodes_to_scores(root_node.child_nodes)  # 訪問回数のリスト

    return boltzman(scores, temperature)  # 合計1に直した訪問回数のリスト、次に選ばれる行動の確率。


def pv_mcts_action(model, temperature=0):  # モンテカルロ木探索で行動選択
    def pv_mcts_action(state):  # 関数に同じ名前の関数が入っていることに注意
        scores = pv_mcts_scores(model, state, temperature)
        return np.random.choice(state.legal_actions(), p=scores)
    return pv_mcts_action


def boltzman(xs, temperature):  # ボルツマン分布
    if temperature == 0:  # 最大値のみ1
        action = np.argmax(xs)
        scores = np.zeros(len(xs))
        scores[action] = 1
        return scores
    else:  # ボルツマン分布でバラつき付加
        xs = [x ** (1 / temperature) for x in xs]
        return [x / sum(xs) for x in xs]

In [ ]:
# デバッグ用
state = State()
model = Dual()
next_action = pv_mcts_action(model, 0)

# ゲーム終了までループ
while True:
    # ゲーム終了時
    if state.is_done():
        break

    # 行動の取得
    action = next_action(state)

    # 次の状態の取得
    state = state.next(action)

    # 文字列表示
    print(state)

---
-o-
---

x--
-o-
---

xo-
-o-
---

xo-
-o-
-x-

xo-
-o-
ox-

xox
-o-
ox-

xox
oo-
ox-

xox
oox
ox-

xox
oox
oxo



### 3.2 リプレイバッファ

In [ ]:
class ReplayBuffer:
    def __init__(self, memory_size):
        self.memory = deque([], maxlen = memory_size)
    
    def append(self, transition):
        self.memory.append(transition)
    
    def sample(self, batch_size):
        batch_indexes = np.random.randint(0, len(self.memory), size=batch_size)
        pieces = np.array([self.memory[index]['piece'] for index in batch_indexes])  # 自分と相手の盤面
        actions = np.array([self.memory[index]['action'] for index in batch_indexes])  # 行動確率
        values = np.array([self.memory[index]['value'] for index in batch_indexes])  # 勝敗
        return {'pieces': pieces, 'actions': actions, 'values': values}

## 4.学習

In [ ]:
n_epochs = 10
lr=0.01
memory_size = 50000
batch_size = 128

SP_GAME_COUNT = 50  # セルフプレイを行うゲーム数（本家は25000）
SP_TEMPERATURE = 1.0  # 訓練時のボルツマン分布の温度

DN_OUTPUT_SIZE = 9

RN_EPOCHS = 200  # 学習回数

EN_GAME_COUNT = 50  # 1評価あたりのゲーム数（本家は400）
EN_TEMPERATURE = 1.0  # テスト時のボルツマン分布の温度


model = Dual()
# 損失関数の設定
# criterion = nn.BCELoss()  # マルチクラス分類用
criterion = nn.CrossEntropyLoss()  # softmax関数と通したあとloglossを計算

# 最適化手法の設定
# optimizer = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 学習率は固定

replay_buffer = ReplayBuffer(memory_size)

if not os.path.exists('./Alpha_pytorch_model/best.pth'):
    torch.save(model.cpu().state_dict(), './Alpha_pytorch_model/best.pth')
    model = model.to(device)


In [ ]:
log_dir = 'logs'
writer = SummaryWriter(log_dir)

In [ ]:
def play(first_action, second_action):
    state = State()
    while True:
        if state.is_done():
            if state.is_lose():
                return [0, 1, 0] if state.is_first_player() else [1, 0, 0]
            else:
                return [0, 0, 1]

        if state.is_first_player():
            action = action0(state)
        else:
            action = action1(state)
        state = state.next(action)

In [ ]:
# action0 = random_action
action0 = alpha_beta_action

best_model = Dual()
best_model.load_state_dict(torch.load('./Alpha_pytorch_model/best.pth'))
action1 = pv_mcts_action(best_model, 0)  # 過去最強のモデル

first_point = np.array([0, 0, 0])
for i in range(50):
    first_point += np.array(play(action0, action1))
print('後手:過去最強 | 先手 {}勝{}敗{}分'.format(first_point[0], first_point[1], first_point[2]))

second_point = np.array([0, 0, 0])
for i in range(50):
    second_point += np.array(play(action1, action0))
print('先手:過去最強 | 後手 {}勝{}敗{}分'.format(second_point[1], second_point[0], second_point[2]))

先手:ランダム, 後手:過去最強 | 先手 49勝1敗0分
先手:過去最強, 後手:ランダム | 後手 1勝48敗1分


In [ ]:
print(len(replay_buffer.memory))

736


In [ ]:
for epoch in range(n_epochs):
    # -------------------------------------------------------------------------------------
    # 最新モデル同士で500回対戦
    # -------------------------------------------------------------------------------------
    for game_count in range(SP_GAME_COUNT):
        history = []

        # 1ゲームの実行
        state = State()

        while True:
            if state.is_done():  # ゲーム終了時
                break

            # アーク評価値による訪問確率とその行動をデータに保存
            scores = pv_mcts_scores(model, state, SP_TEMPERATURE)

            policies = [0] * DN_OUTPUT_SIZE
            for action, policy in zip(state.legal_actions(), scores):
                policies[action] = policy

            history.append([[state.pieces, state.enemy_pieces], policies, None])

            # scoresに基づいて行動し、次の状態を取得
            action = np.random.choice(state.legal_actions(), p=scores)
            state = state.next(action)
        
        if state.is_lose():
            value = -1 if state.is_first_player() else 1
        else:
            value = 0

        for i in range(len(history)):
            history[i][2] = value  # 価値はエピソードの初めまで、割引無しで伝搬させる。
            value = -value  # 配置が交互に入れ替わっているので価値も反転させる

        # 時系列を無視してリプレイバッファに保存
        for i in range(len(history)):
            transition = {'piece': history[i][0], 'action': history[i][1], 'value': history[i][2]}
            replay_buffer.append(transition)


        print('\rSelfPlay {}/{}'.format(game_count+1, SP_GAME_COUNT), end='')
    print('')

    # -------------------------------------------------------------------------------------
    # パラメータ更新部
    # 価値：状態から勝ち負けを予測する。
    # 方策：価値を用いたアーク評価値による訪問確率と、方策が一致するように学習していく。
    # -------------------------------------------------------------------------------------
    losses = []
    for update_step in range(RN_EPOCHS):
        # バッファから読み込む
        batch = replay_buffer.sample(batch_size)
        xs, y_policies, y_values = batch['pieces'], batch['actions'], batch['values']

        # データの前処理
        xs = torch.as_tensor(xs, dtype=torch.float, device=device).view(-1, 18)  # Fratten
        y_policies = torch.as_tensor(y_policies, dtype=torch.float, device=device)
        y_values = torch.as_tensor(y_values, dtype=torch.float, device=device).view(-1, 1)

        # 学習
        p, v = model(xs)

        loss_p = criterion(p.cpu(), y_policies).to(device)
        loss_v = criterion(v.cpu(), y_values).to(device)
        loss = loss_p + loss_v  # lossは足し算
        loss.backward()  # 誤差伝搬
        optimizer.step()  # Adamでパラメータ更新
        losses.append(loss.cpu().detach().numpy())

        print('\r%3d /%3d' % (update_step+1, RN_EPOCHS), end='')
    print('')
    print('エポック: %d  loss: %lf' %(epoch+1, np.average(losses)))
    writer.add_scalar('train loss', loss.item(), epoch+1)

    # -------------------------------------------------------------------------------------
    # 新パラメータ評価部
    # 色んな方法がある。「対ランダムにどれだけの割合で勝てるか」「過去の自分に勝てるか」
    # しかし三すくみみたいになってしまう可能性
    # 何なら、温度もハイパーパラメータみたいにしたい。学習時も相手と自分で違うモデルを作成すべき？
    # -------------------------------------------------------------------------------------

    # 実際に検証したいときの、モデルの読み込み
    action0 = pv_mcts_action(model, EN_TEMPERATURE)

    best_model = Dual()
    best_model.load_state_dict(torch.load('./Alpha_pytorch_model/best.pth'))
    action1 = pv_mcts_action(best_model, EN_TEMPERATURE)  # 過去最強のモデル

    first_point = np.array([0, 0, 0])
    for i in range(EN_GAME_COUNT):
        first_point += np.array(play(action0, action1))
    print('先手:最新, 後手:過去最強 | 先手 {}勝{}敗{}分'.format(first_point[0], first_point[1], first_point[2]))

    second_point = np.array([0, 0, 0])
    for i in range(EN_GAME_COUNT):
        second_point += np.array(play(action1, action0))
    print('先手:過去最強, 後手:最新 | 後手 {}勝{}敗{}分'.format(second_point[1], second_point[0], second_point[2]))

    if first_point[0]+second_point[1] > first_point[1]+second_point[0]:
        torch.save(model.cpu().state_dict(), './Alpha_pytorch_model/best.pth')
        model = model.to(device)
        print('保存完了！')


SelfPlay 50/50
200 /200
エポック: 1  loss: 2.107130
先手:最新, 後手:過去最強 | 先手 42勝5敗3分
先手:過去最強, 後手:最新 | 後手 5勝42敗3分
SelfPlay 50/50
200 /200
エポック: 2  loss: 1.927551
先手:最新, 後手:過去最強 | 先手 31勝17敗2分
先手:過去最強, 後手:最新 | 後手 14勝34敗2分
SelfPlay 50/50
200 /200
エポック: 3  loss: 1.863579
先手:最新, 後手:過去最強 | 先手 16勝17敗17分
先手:過去最強, 後手:最新 | 後手 12勝16敗22分
SelfPlay 50/50
200 /200
エポック: 4  loss: 1.838816
先手:最新, 後手:過去最強 | 先手 10勝37敗3分
先手:過去最強, 後手:最新 | 後手 39勝9敗2分
保存完了！
SelfPlay 50/50
200 /200
エポック: 5  loss: 1.801143
先手:最新, 後手:過去最強 | 先手 23勝22敗5分
先手:過去最強, 後手:最新 | 後手 21勝28敗1分
SelfPlay 50/50
200 /200
エポック: 6  loss: 1.822034
先手:最新, 後手:過去最強 | 先手 41勝6敗3分
先手:過去最強, 後手:最新 | 後手 3勝46敗1分
SelfPlay 50/50
200 /200
エポック: 7  loss: 1.797927
先手:最新, 後手:過去最強 | 先手 39勝8敗3分
先手:過去最強, 後手:最新 | 後手 8勝42敗0分
SelfPlay 50/50
200 /200
エポック: 8  loss: 1.832163
先手:最新, 後手:過去最強 | 先手 42勝8敗0分
先手:過去最強, 後手:最新 | 後手 9勝41敗0分
保存完了！
SelfPlay 50/50
200 /200
エポック: 9  loss: 1.834982
先手:最新, 後手:過去最強 | 先手 23勝27敗0分
先手:過去最強, 後手:最新 | 後手 16勝32敗2分
SelfPlay 50/50
200 /200
エポック: 10  loss: 1